# Deep Semantic Search for Hyperpersonalized Recommendations

## Setup

You can get the original code and data from zach-blumenfeld's [Neo4j Generative AI Workshop](https://github.com/neo4j-product-examples/genai-workshop/tree/main)

In [ ]:
%%capture
%pip install sentence_transformers langchain openai tiktoken python-dotenv gradio graphdatascience altair
%pip install "vegafusion[embed]"

In [ ]:
from graphdatascience import GraphDataScience
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from langchain.embeddings import OpenAIEmbeddings, BedrockEmbeddings, SentenceTransformerEmbeddings

### Setup Credentials and Environment Variables

To make this easy, you can write the credentials and env variables directly into the below cell

If you like you can use an environments file instead by copying `ws.env.template` to `ws.env` and filling credentials and variables in there. This is a best practice for the future, but fine to skip for this workshop.

In [ ]:
# You can skip this if not using a ws.env file
if os.path.exists('ws.env'):
    load_dotenv('ws.env', override=True)

    # Neo4j
    NEO4J_URI = os.getenv('NEO4J_URI')
    NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
    NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
    AURA_DS = False

    # AI
    EMBEDDING_MODEL = 'openai'
    LLM = 'gpt-3.5'

### Connect to Neo4j

In [ ]:
# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    aura_ds=AURA_DS)

# Necessary if you enabled Arrow on the db - this is true for AuraDS
gds.set_database("hnm")

## Vector Search
In this Section We will build Text Embeddings of Product and demonstrate how to leverage the Neo4j vector index for vector search.

### Creating Text Embeddings

In [ ]:
def load_embedding_model(embedding_model_name: str):
    if embedding_model_name == "openai":
        embeddings = OpenAIEmbeddings()
        dimension = 1536
        print("Embedding Model: openai")
    elif embedding_model_name == "aws":
        embeddings = BedrockEmbeddings()
        dimension = 1536
        print("Embedding Model: aws")
    else:
        embeddings = SentenceTransformerEmbeddings(
            model_name="all-MiniLM-L6-v2", cache_folder="/embedding_model")
        print("Embedding Model: sentence transformer")
        dimension = 384
    return embeddings, dimension

In [ ]:
embedding_model, dimension = load_embedding_model(EMBEDDING_MODEL)

### Vector Search Using Cypher

In [ ]:
#search_prompt = 'denim jeans, loose fit, high-waist'
search_prompt = input() #'Oversized Sweaters'

In [ ]:
query_vector = embedding_model.embed_query(search_prompt)
print(f'query vector length: {len(query_vector)}')
print(f'query vector sample: {query_vector[:10]}')

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

In [ ]:
gds.run_cypher('''
CALL db.index.vector.queryNodes("product-text-embeddings", 10, $queryVector)
YIELD node AS product, score
RETURN product.productCode AS productCode,
    product.text AS text,
    score
''', params={'queryVector': query_vector})

### Vector Search Using Langchain

We can also do this with langchain which is a recommended approach going forward.  To do this we use the Neo4jVector class and call the method to sert it up from an existing index in the graph.

In [ ]:
from langchain.vectorstores.neo4j_vector import Neo4jVector

In [ ]:
kg_vector_search = Neo4jVector.from_existing_index(
    embedding=embedding_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database='hnm',
    index_name='product-text-embeddings')

In [ ]:
res = kg_vector_search.similarity_search(search_prompt, k=10)
res

In [ ]:
# Visualize as a dataframe
pd.DataFrame([{'document': d.page_content} for d in res])

### Try Yourself

In [ ]:
res = kg_vector_search.similarity_search('red sweather', k=10)
pd.DataFrame([{'document': d.page_content} for d in res])

## Semantic Search with Context
Using Explicit Relationships in enterprise data


Above we see how you can use the vector index to find semantic similar products in user searches.  but there is a rich graph full of other information in it. Lets leverage our knowledge graph to make this better

An important piece of information expressed in this graph, but not directly in the documents, is customer purchasing behavior.  We can use A Cypher Query to make recommendations without any document behavior. this is similar to collaborative filtering but generalized to purchase history (not necessarily rating based)

#### Example Purchase History

Consider the below customer

In [ ]:
CUSTOMER_ID = "daae10780ecd14990ea190a1e9917da33fe96cd8cfa5e80b67b4600171aa77e0"
print('Customer Purchase History')
gds.run_cypher('''
    MATCH(c:Customer {customerId: $customerId})-[:PURCHASED]->(:Article)
    -[:VARIANT_OF]->(p:Product)
    RETURN p.productCode AS productCode,
        p.prodName AS prodName,
        p.productTypeName AS productTypeName,
        p.garmentGroupName AS garmentGroupName,
        p.detailDesc AS detailDesc,
        count(*) AS purchaseCount
    ORDER BY purchaseCount DESC
''', params={'customerId': CUSTOMER_ID})

#### Graph Patterns For Retrieval Query

In [ ]:
# This is the example Pattern we can use to predict likely customer preferences based on collaborative behavior
gds.run_cypher('''
    MATCH(c:Customer {customerId: $customerId})-[:PURCHASED]->(:Article)
    <-[:PURCHASED]-(:Customer)-[:PURCHASED]->(:Article)
    -[:VARIANT_OF]->(p:Product)
    RETURN p.productCode AS productCode,
        p.prodName AS prodName,
        p.productTypeName AS productTypeName,
        p.garmentGroupName AS garmentGroupName,
        p.detailDesc AS detailDesc,
        count(*) AS score
    ORDER BY score DESC LIMIT 10
''', params={'customerId': CUSTOMER_ID})

In [ ]:
# This is the example Pattern we can use to predict likely customer preferences based on collaborative behavior
# Finds products that match the search and has been purchased by customers with same purchase history as customer XXX

kg_personalized_search = Neo4jVector.from_existing_index(
    embedding=embedding_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database='hnm',
    index_name='product-text-embeddings',
    retrieval_query=f"""
    WITH node AS product, score AS searchScore

    OPTIONAL MATCH(product)<-[:VARIANT_OF]-(:Article)<-[:PURCHASED]-(:Customer)
    -[:PURCHASED]->(a:Article)<-[:PURCHASED]-(:Customer {{customerId: '{CUSTOMER_ID}'}})

    WITH count(a) AS purchaseScore, product.text AS text, searchScore, product.productCode AS productCode
    RETURN text,
        (1+purchaseScore)*searchScore AS score,
        {{productCode: productCode, purchaseScore:purchaseScore, searchScore:searchScore}} AS metadata
    ORDER BY purchaseScore DESC, searchScore DESC LIMIT 15
    """)

In [ ]:
res = kg_personalized_search.similarity_search(search_prompt, k=100)

# Visualize as a dataframe
pd.DataFrame([{'productCode': d.metadata['productCode'],
               'document': d.page_content,
               'searchScore': d.metadata['searchScore'],
               'purchaseScore': d.metadata['purchaseScore']} for d in res])

## KG Powered Inference for AI

We saw before how could use graph pattern matching to personalize search and make it more relevant.

Graph pattern matching is very power and can work well in a lot of scenarios.

In addition to this, we also have Graph Data Science, which can allow as to enrich the current Knowledge graph with machine learning, that can
1. Provide addition information to improve relevancy of search results at scale
2. Provide additional inferences to GenAI

We will show an example of how this works using Node Embedding and K-Nearest Neighbor algorithms



### Graph Embedding

In [ ]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

In [ ]:
def clear_all_graphs():
    g_names = gds.graph.list().graphName.tolist()
    for g_name in g_names:
        g = gds.graph.get(g_name)
        g.drop()

#### Clear Past Analysis (If rerunning this Notebook)

In [ ]:
clear_all_graphs()

In [ ]:
gds.run_cypher('''
    MATCH(:Article)-[r:CUSTOMERS_ALSO_LIKE]->()
    CALL {
        WITH r
        DELETE r
    } IN TRANSACTIONS OF 1000 ROWS
    ''')

#### Apply Fast Random Projection Node Embedding

First, apply a graph projection to structure the portion of the graph we need in an optimized in-memory format for graph ML.

In [ ]:
%%time

# graph projection
gds.run_cypher('''
   MATCH (a1:Article)<-[:PURCHASED]-(:Customer)-[:PURCHASED]->(a2:Article)
   WITH gds.graph.project("proj", a1, a2,
       {sourceNodeLabels: labels(a1),
       targetNodeLabels: labels(a2),
       relationshipType: "COPURCHASE"}) AS g
   RETURN g.graphName
   ''')

g = gds.graph.get("proj")

Next, we will generate node embeddings for similarity calculation.  In this case, we will use FastRP (Fast Random Projection) which is a fast, scalable, and robust embedding algorithm. FastRP calculates embeddings using probabilistic sampling and linear algebra.

In [ ]:
%%time
# embeddings (writing back Article embeddings in case we want to introspect later)
gds.fastRP.mutate(g, mutateProperty='embedding', embeddingDimension=128, randomSeed=7474, concurrency=4, iterationWeights=[0.0, 1.0, 1.0])
gds.graph.writeNodeProperties(g, ['embedding'], ['Article'])

#### Explore Node Embeddings

In [ ]:
graph_emb_df = gds.run_cypher('''
MATCH (p:Product)<-[:VARIANT_OF]-(a:Article)-[:FROM_DEPARTMENT]-(d)
RETURN a.articleId AS articleId,
    p.prodName AS productName,
    p.productTypeName AS productTypeName,
    d.departmentName AS departmentName,
    d.sectionName AS sectionName,
    p.detailDesc AS detailDesc,
    a.embedding AS embedding
''')

In [ ]:
graph_emb_df[:3]

In [ ]:
# Takes 30sec to run
from sklearn.manifold import TSNE

df = graph_emb_df.copy()
filtered_node_df = df[df.embedding.apply(lambda x: np.count_nonzero(x) > 0)].reset_index(drop=True)
# instantiate the TSNE model
tsne = TSNE(n_components=2, random_state=7474, init='random', learning_rate="auto")
# Use the TSNE model to fit and output a 2-d representation
E = tsne.fit_transform(np.stack(filtered_node_df['embedding'], axis=0))

coord_df = pd.concat([filtered_node_df, pd.DataFrame(E, columns=['x', 'y'])], axis=1)
coord_df

In [ ]:
import altair as alt
from sklearn.manifold import TSNE

alt.data_transformers.disable_max_rows()
chart = alt.Chart(coord_df.sample(n=5000, random_state=7474)).mark_circle(size=60).encode(
    x='x',
    y='y',
    tooltip=['productName', 'productTypeName', 'departmentName' , 'sectionName', 'detailDesc']
).properties(title="Article Embedding (2D Representation)", width=750, height=700)

chart = chart.configure_axis(titleFontSize=20)
chart.configure_legend(labelFontSize = 20)
chart

### K-Nearest Neighbors (KNN) Relationships

Finally, we can do our similarity inference with K-Nearest Neighbor (KNN) and write back to the graph.
We will use a slightly low cutoff of 0.75 similarity score to extend the result size for exploration.  We can provide a higher cutoff at query time if needed.

In [ ]:
%%time
# KNN
_ = gds.knn.write(g, nodeProperties=['embedding'], nodeLabels=['Article'],
                  writeRelationshipType='CUSTOMERS_ALSO_LIKE', writeProperty='score',
                  sampleRate=1.0, initialSampler='randomWalk', concurrency=1, similarityCutoff=0.75, randomSeed=7474)
_

In [ ]:
# clear graph projection once done
g.drop()

### Tailored Recommendations from Search

Let's construct a KG store to retrieve recommendations based on search

In [ ]:
# Based on a search, what other product should we recommend?
kg_search_recommendations = Neo4jVector.from_existing_index(
    embedding=embedding_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database='hnm',
    index_name='product-text-embeddings',
    retrieval_query="""
    WITH node as searchProduct, score as searchScore
    MATCH(searchProduct)<-[:VARIANT_OF]-(:Article)-[r:CUSTOMERS_ALSO_LIKE]->(:Article)-[:VARIANT_OF]-(product)
    WITH  product, searchScore, sum(r.score*searchScore) AS recommenderScore
    RETURN product.text AS text,
    recommenderScore AS score,
    {productCode: product.productCode, productType: product.productTypeName, recommenderScore:recommenderScore} AS metadata
    ORDER BY score DESC LIMIT 100
    """
)

In [ ]:
res = kg_search_recommendations.similarity_search(search_prompt, k=100)

# Visualize as a dataframe
pd.DataFrame([{'productCode': d.metadata['productCode'],
               'productType':d.metadata['productType'],
               'document': d.page_content,
               'recommenderScore': d.metadata['recommenderScore']} for d in res])

### Personalized Recommendations

N ow lets look at personalized recommendations for a bit.  To keep things simple we will base this just on purchase history, not search, though we could do both if we wanted to (similar to what we did in the above Semantic Search with context section)

First, we will start by creating a Neo4jGraph object which we can then query. This is different from the vec tor based retrievers above

In [ ]:
from langchain.graphs import Neo4jGraph

kg = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database='hnm')

In [ ]:
# Looking at this customer purchase history, what ptoduct could be recommended to him?
res = kg.query('''
    MATCH(:Customer {customerId:$customerId})-[:PURCHASED]->(:Article)
    -[r:CUSTOMERS_ALSO_LIKE]->(:Article)-[:VARIANT_OF]->(product)
    RETURN product.productCode AS productCode,
        product.prodName AS prodName,
        product.productTypeName AS productType,
        product.text AS document,
        sum(r.score) AS recommenderScore
    ORDER BY recommenderScore DESC LIMIT $k
    ''', params={'customerId': CUSTOMER_ID, 'k':15})

#visualize as dataframe. result is list of dict
pd.DataFrame(res)

We could also make it based of the latest purchases.  For example consider the last purchase for the customer

In [ ]:
# last two purchases
pd.DataFrame(kg.query('''
    MATCH(:Customer {customerId:$customerId})-[t:PURCHASED]->(:Article)-[:VARIANT_OF]->(product)
    RETURN product.productCode AS productCode,
        product.prodName AS prodName,
        product.productTypeName AS productType,
        product.text AS document,
        t.tDat as purchaseDate
    ORDER BY purchaseDate DESC LIMIT $k
    ''', params={'customerId': CUSTOMER_ID, 'k':3}))

In [ ]:
# Based on the last 20 purchase, what product should we recommedn to this particular customer?
res = kg.query('''
    MATCH(:Customer {customerId:$customerId})-[t:PURCHASED]->(a:Article)
    WITH a, t.tDat as purchaseDate
    ORDER BY purchaseDate DESC LIMIT $lastNPurchases
    MATCH(a)-[r:CUSTOMERS_ALSO_LIKE]->(:Article)-[:VARIANT_OF]->(product)
    RETURN product.productCode AS productCode,
        product.prodName AS prodName,
        product.productTypeName AS productType,
        product.text AS document,
        sum(r.score) AS recommenderScore
    ORDER BY recommenderScore DESC LIMIT $k
    ''', params={'customerId': CUSTOMER_ID, 'lastNPurchases':20, 'k':15})

#visualize as dataframe. result is list of dict
pd.DataFrame(res)

The same could be done with other filters, such as on transaction date.

## LLM For Generating Grounded Content

Let's use an LLM to automatically generate content for targeted marketing campaigns grounded with our knowledge graph using the above tools.
Here is a quick example for generating promotional messages. but you can create all sorts of content with this!

For our first message, let's consider a scenario where a user recently searched for products, but perhaps didn't commit to a purchase yet. We now want to send a message to promote relevant products.

In [ ]:
# Import relevant libraries
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

In [ ]:
#load LLM

def load_llm(llm_name: str):
    if llm_name == "gpt-4":
        print("LLM: Using GPT-4")
        return ChatOpenAI(temperature=0, model_name="gpt-4", streaming=True)
    elif llm_name == "gpt-3.5":
        print("LLM: Using GPT-3.5")
        return ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)
    elif llm_name == "claudev2":
        print("LLM: ClaudeV2")
        return BedrockChat(
            model_id="anthropic.claude-v2",
            model_kwargs={"temperature": 0.0, "max_tokens_to_sample": 1024},
            streaming=True,
        )
    print("LLM: Using GPT-3.5")
    return ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)


llm = load_llm(LLM)

### Create Knowledge Graph Stores for Retrieval
To ground our content Generation we need to define retrievers to pull information from our knowledge graph.  Let's make two stores:
1. Personalized Search Retriever (`kg_personalized_search`): Based on recent customer searches and purchase history, pull relevant products
2. Recommendations retriever (`kg_recommendations`): Based on recent customer searches, what else may we recommend to them?

In [ ]:
# This will be a function so we can change per customer id
# We will use a mock URL for our sources in the metadata
def kg_personalized_search_gen(customer_id):
    return Neo4jVector.from_existing_index(
        embedding=embedding_model,
        url=NEO4J_URI,
        username=NEO4J_USERNAME,
        password=NEO4J_PASSWORD,
        database='hnm',
        index_name='product-text-embeddings',
        retrieval_query=f"""
        WITH node AS product, score AS searchScore

        OPTIONAL MATCH(product)<-[:VARIANT_OF]-(:Article)<-[:PURCHASED]-(:Customer)
        -[:PURCHASED]->(a:Article)<-[:PURCHASED]-(:Customer {{customerId: '{customer_id}'}})
        WITH count(a) AS purchaseScore, product, searchScore
        RETURN product.text + '\nurl: ' + 'https://representative-domain/product/' + product.productCode  AS text,
            (1.0+purchaseScore)*searchScore AS score,
            {{source: 'https://representative-domain/product/' + product.productCode}} AS metadata
        ORDER BY purchaseScore DESC, searchScore DESC LIMIT 5

    """
    )

In [ ]:
# Use the same tailored search recommendations as above but with a smaller limit
kg_recommendations_bot1 = Neo4jVector.from_existing_index(
    embedding=embedding_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database='hnm',
    index_name='product-text-embeddings',
    retrieval_query="""
    WITH node as searchProduct, score as searchScore
    MATCH(searchProduct)<-[:VARIANT_OF]-(:Article)-[r:CUSTOMERS_ALSO_LIKE]->(:Article)-[:VARIANT_OF]-(product)
    WITH  product, searchScore, sum(r.score*searchScore) AS recommenderScore
    RETURN product.text + '\nurl: ' + 'https://representative-domain/product/' + product.productCode  AS text,
    recommenderScore AS score,
    {source: 'https://representative-domain/product/' + product.productCode} AS metadata
    ORDER BY score DESC LIMIT 5
    """
)

### Prompt Engineering
Now let's define our prompts. We will combine two together:
1. A system prompt which, in this case tells the LLM how to generated the message
2. Human prompt: In this case just wraps the search prompt entered by the customer

This will allow us to pass the customer search to the retrievers, but then also to the LLM for addition context when drafting the message.

In [ ]:
general_system_template = '''
You are a personal assistant named Sally for a fashion, home, and beauty company called HRM.
write an email to {customerName}, one of your customers, to promote and summarize products relevant for them given the current season / time of year: {timeOfYear} .
Please only mention the Products listed below. Do not come up with or add any new products to the list.
Each product description comes with a "url" field. make sure to link to the url with descriptive name text for each product so the customer can easily find them.

---
# Relevant Products:
{searchProds}

# Customer May Also Be Interested In:
{recProds}
---
'''
general_user_template = "{searchPrompt}"
messages = [
    SystemMessagePromptTemplate.from_template(general_system_template),
    HumanMessagePromptTemplate.from_template(general_user_template),
]
prompt = ChatPromptTemplate.from_messages(messages)

### Create a Chain
Now let's put a chain together that will leverage the retrievers, prompts, and LLM model. This is where Langchain shines, putting RAG together in a simple way.

In addition to the personalized search and recommendations context, we will allow for som other parameters

1. `customerName`: Ordinarily this will be pulled from Neo4j, but it has been scrubbed from the data for obvious reasons so we will provide our own name here.
2. `timeOfYear`: The time of year as a date, season, month, etc. the LLM can tailor the language appropriately.

You can potentially add other creative parameters here to help the LLM write relevant messages.


In [ ]:
# Helper function
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

def chain_gen(customer_id):
    return ({'searchProds': (lambda x:x['searchPrompt']) | kg_personalized_search_gen(customer_id).as_retriever(search_kwargs={"k": 100}) | format_docs,
              'recProds': (lambda x:x['searchPrompt']) | kg_recommendations_bot1.as_retriever(search_kwargs={"k": 5}) | format_docs,
              'customerName': lambda x:x['customerName'],
              'timeOfYear': lambda x:x['timeOfYear'],
              "searchPrompt":  lambda x:x['searchPrompt']}
             | prompt
             | llm
             | StrOutputParser())

### Examples Runs

In [ ]:
chain = chain_gen(CUSTOMER_ID)

In [ ]:
print(chain.invoke({'searchPrompt':search_prompt, 'customerName':'Alex Smith', 'timeOfYear':'Dec, 2023'}))

In [ ]:
print(chain.invoke({'searchPrompt':"western boots", 'customerName':'Alex Smith', 'timeOfYear':'Dec, 2023'}))

Feel free to experiment and try more!

### Demo App
Now lets use the above tools to create a demo app with Gradio.  We will need to make a couple more functions, but otherwise easy to fire up from a Notebook!

In [ ]:
# Create a means to generate and cache chains...so we can quickly try different customer ids
personalized_search_chain_cache = dict()
def get_chain(customer_id):
    if customer_id in personalized_search_chain_cache:
        return personalized_search_chain_cache[customer_id]
    chain = chain_gen(customer_id)
    personalized_search_chain_cache[customer_id] = chain
    return chain

In [ ]:
import gradio as gr

def message_generator(*x):
    chain = get_chain(x[0])
    return chain.invoke({'searchPrompt':x[3], 'customerName':x[2], 'timeOfYear': x[1]})

customer_id = gr.Textbox(value=CUSTOMER_ID, label="Customer ID")
time_of_year = gr.Textbox(value="Dec, 2023", label="Time Of Year")
search_prompt = gr.Textbox(value='Oversized Sweaters', label="Customer Interests(s)")
customer_name = gr.Textbox(value='Alex Smith', label="Customer Name")
message_result = gr.Markdown( label="Message")

demo = gr.Interface(fn=message_generator,
                    inputs=[customer_id, time_of_year, customer_name, search_prompt],
                    outputs=message_result,
                    title="🪄 Message Generator 🥳")
demo.launch(share=True, debug=True)

### Demo App - Directly to Recommendations
There are lots of different ways we can configure this.  Let's try a shorter version that cuts right to personalized recommendations and makes an in season pun.

First we will create a function to retrieve based off our personalized recommendations example

In [ ]:
def kg_recommendations_app2(customer_id, k=30):
    res = kg.query("""
    MATCH(:Customer {customerId:$customerId})-[:PURCHASED]->(:Article)
    -[r:CUSTOMERS_ALSO_LIKE]->(:Article)-[:VARIANT_OF]->(product)
    RETURN product.text + '\nurl: ' + 'https://representative-domain/product/' + product.productCode  AS text,
        sum(r.score) AS recommenderScore
    ORDER BY recommenderScore DESC LIMIT $k
    """, params={'customerId': customer_id, 'k':k})

    return "\n\n".join([d['text'] for d in res])

In [ ]:
# test out
print(kg_recommendations_app2(CUSTOMER_ID))

Next we re-define our prompt

In [ ]:
general_system_template_app2 = '''
You are a personal assistant named Sally for a fashion, home, and beauty company called HRM.
write an email to {customerName}, one of your customers, to promote and summarize products that fasionably pair with what they searched for given the current season / time of year: {timeOfYear}.
Make an in-season pun too!
Please only choose from the Products listed below. Choose no more than 5. Do not come up with or add any new products to the list.
Each product description comes with a "url" field. make sure to link to the url with descriptive name text for each product so the customer can easily find them.

---
# Relevant Products:
{recProds}
---
'''

general_user_template_app2 = '''Something that goes with {searchPrompt}'''
messages_app2 = [
    SystemMessagePromptTemplate.from_template(general_system_template_app2),
    HumanMessagePromptTemplate.from_template(general_user_template_app2),
]
prompt_app2 = ChatPromptTemplate.from_messages(messages_app2)

In [ ]:
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda

chain_app2 = ({'recProds': itemgetter('customerId') |  RunnableLambda(kg_recommendations_app2),
             'customerName': lambda x:x['customerName'],
             'timeOfYear': lambda x:x['timeOfYear'],
             "searchPrompt":  lambda x:x['searchPrompt']}
            | prompt_app2
            | llm
            | StrOutputParser())

In [ ]:
print(chain_app2.invoke({'customerId':CUSTOMER_ID, 'searchPrompt':"western boots", 'customerName':'Alex Smith', 'timeOfYear':'Nov, 2023'}))

In [ ]:
import gradio as gr

def message_generator_app2(*x):
    return chain_app2.invoke({'searchPrompt':x[3],
                              'customerName':x[2],
                              'timeOfYear': x[1],
                              'customerId': x[0]})

customer_id = gr.Textbox(value=CUSTOMER_ID, label="Customer ID")
time_of_year = gr.Textbox(value="Nov, 2023", label="Time Of Year")
customer_name = gr.Textbox(value='Alex Smith', label="Customer Name")
search_prompt = gr.Textbox(value='Oversized Sweaters', label="Customer Interests(s)")
message_result = gr.Markdown( label="Message")

demo = gr.Interface(fn=message_generator_app2,
                    inputs=[customer_id, time_of_year, customer_name, search_prompt],
                    outputs=message_result,
                    title="🪄 Message Generator - Recommendations and Puns 🥳")
demo.launch(share=True, debug=True)

## Wrap Up